<a href="https://colab.research.google.com/github/Will-est/PPO-From-Scratch/blob/main/PPO_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import statments
import gymnasium as gym
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
import torch
import torch.nn as nn
import torch.distributions as distributions

ModuleNotFoundError: No module named 'stable_baselines3'

In [ ]:
# Hper parameters

initial_learning_rate = 0.1
env_id = "LunarLander-v2" # Changed to v2 for consistency with common practice
clipping_coef = 0.1
num_envs = 4
steps = 1e5
middle_layer_size = 64 # Define the size of the middle layer
batch_size = 256 # Define a batch size for reshaping

# Conventional Vectoriz3ed Environment wrapper
def make_env(env_id):
    def _init():
        env = gym.make(env_id)
        # Optional: Add wrappers here if needed
        return env
    return _init

# Initialize info dictionary or maybe a list of dictionaries where each entry contains the mean reward, loss, number of steps, learning rate
info = []

# Agent definition

class Agent(nn.Module):
  def __init__(self, observation_space_shape, action_space_size, middle_layer_size) -> None:
      super().__init__() # Corrected syntax

      # Actor/Policy
      self.actor = nn.Sequential(
          nn.Linear(observation_space_shape, middle_layer_size),
          nn.ReLU(),
          nn.Linear(middle_layer_size, action_space_size),
          nn.Softmax(dim=-1)
          ) # Added dim=-1 to softmax

      # Critic/Advantage NN //might need another activation function at the end. Also at somepoint I am going to need to get the values of the S primes which Idk when I am going to do that
      self.critic = nn.Sequential(
          nn.Linear(observation_space_shape, middle_layer_size),
          nn.ReLU(),
          nn.Linear(middle_layer_size, 1), # Output size of 1 for the value function
          )

  def predict(self, x): # Corrected self parameter
    action_probs = self.actor(x)
    act_dist = distributions.Categorical(action_probs) # Create a categorical distribution
    action = act_dist.sample()

    entropy = act_dist.entropy() # Calculate entropy

    value_logits = self.critic(x)

    # return entropy, probabilies, and sampled action
    return (entropy, act_dist, action, value_logits) # Return entropy, probabilities, and a sampled action

if __name__ == "__main__": # Corrected __main__

  # initilize invironment
  envs = gym.vector.AsyncVectorEnv([make_env(env_id) for i in range(num_envs)])

  # Get observation and action space dimensions
  observation_space_shape = envs.single_observation_space.shape[0] # Assuming flat observation space
  action_space_size = envs.single_action_space.n # Assuming discrete action space

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # Removed args.cuda

  #initialize the Agent
  agent  = Agent(observation_space_shape, action_space_size, middle_layer_size).to(device) # Pass dimensions and move to device
  # put things onto gpu
  # ALGO Logic: Storage setup
  # These should be lists to store data from each rollout

  # Initialize tensors with appropriate shapes
  -


  # initializes the observation, done, the time, and the step
  start_time = time.time()
  next_obs = torch.Tensor(envs.reset()[0]).to(device) # Corrected envs.reset()
  next_done =  torch.zeros((num_envs,)).to(device)
  global_step = 0
  step = 0 # Initialize step counter for batch

  # define training regime
  for i in range(int(steps)): # Cast steps to int

    # get actions, observations, rewards, and dones
    with torch.no_grad(): # Added no_grad for inference
        entropy, action_dist, action, values_ = agent.predict(next_obs) # Renamed values to values_ to avoid conflict

    # Store data in tensors at the current step
    obs[step] = next_obs
    actions[step] = action
    logprobs[step] = action_dist.log_prob(action) # Calculate log probability
    values[step] = values_.squeeze(-1) # Remove the last dimension of size 1
    dones[step] = next_done

    next_obs, rewards_, next_done, infos =  envs.step(action.cpu().numpy()) # env step and move action to cpu
    rewards[step] = torch.Tensor(rewards_).to(device) # Store rewards as tensor
    next_obs = torch.Tensor(next_obs).to(device)
    next_done = torch.Tensor(next_done).to(device)

    global_step += num_envs # Update global step
    step += 1 # Increment step counter

    # If batch is full, perform training update (This is a placeholder)
    if step == batch_size:
        # Reshape the collected tensors
        b_obs = obs.reshape((-1, observation_space_shape))
        b_actions = actions.reshape((-1,)) # Reshape actions to be flat
        b_logprobs = logprobs.reshape((-1,))
        b_rewards = rewards.reshape((-1,))
        b_dones = dones.reshape((-1,))
        b_values = values.reshape((-1,))

        # This is where your training update logic would go,
        # using the reshaped tensors (b_obs, b_actions, etc.)

        # Reset step counter and tensors for the next batch
        step = 0
        obs = torch.zeros((batch_size, num_envs, observation_space_shape)).to(device)
        actions = torch.zeros((batch_size, num_envs)).to(device)
        logprobs = torch.zeros((batch_size, num_envs)).to(device)
        rewards = torch.zeros((batch_size, num_envs)).to(device)
        dones = torch.zeros((batch_size, num_envs)).to(device)
        values = torch.zeros((batch_size, num_envs)).to(device)


    # outer loop: rollouts, at each iteration of the larger loop do one step for your policy and update your value function make sure to clip both the policy and value function. ALso include an entropy term

    # inner loop: train value function, step through environment, record rewards, ends, log probs, etc.

    # This part of the code needs to be implemented for the training loop (calculating advantages, updating networks, etc.)
    # This is a placeholder and would typically involve:
    # 1. Calculating advantages/returns
    # 2. Calculating policy and value losses
    # 3. Performing backpropagation and optimizer steps

  envs.close() # Close the environment

SyntaxError: invalid syntax (ipython-input-6-1752336507.py, line 24)

In [ ]:
# prompt: write code to get the current time

import time

print(time.time())

1752440009.3622656


In [ ]:
!pip install stable_baselines3 gymnasium